# Restaurants from delivery sites & reviews

Restaurants are identified by names, similar names are not yet merged.  

In [1]:
# python version
import sys
print "system_info: %s"%sys.version
# current working directory
import os
print "path_info: %s"%os.getcwd() 

system_info: 2.7.14 |Anaconda custom (64-bit)| (default, Oct 27 2017, 18:21:12) 
[GCC 7.2.0]
path_info: /home/yueliu/Desktop/workspace_yue/Documentation_201712


In [2]:
import myfunctions as f
print f.today()

20180207


## 1. Restaurants from reviews
    attributes
>     1. Burpple review locs
    2. Food items mentioned in reviews: (sorted list of food_name & frequency mentioned in reviews)

In [3]:
review_path = "/home/yueliu/Desktop/workspace_yue"
file_name = "burpple__review__clean__20180205.p"    
file_name = os.path.join(review_path, file_name)
df_retrived = f.retrive_file(file_name)

retrived: /home/yueliu/Desktop/workspace_yue/burpple__review__clean__20180205.p


In [4]:
print df_retrived.shape
print df_retrived.columns.tolist()
df_retrived.head(1)

(323104, 10)
['loc', 'title', 'body', 'restaurant.name', 'review_text', 'hashtags', 'mentions', 'dollars', 'scores', 'urls']


,loc,title,body,restaurant.name,review_text,hashtags,mentions,dollars,scores,urls
0,burppleinitial/review/--0EG2cp,Sicilian Lemon Creme andCaramelized Red Mixed ...,,OSO Ristorante,Sicilian Lemon Creme andCaramelized Red Mixed ...,"(#igsg, #igers, #igfood, #yum, #yummy, #instaf...",(),(),(),()


In [5]:
restaurants = df_retrived.groupby('restaurant.name')['loc'].apply(list).to_frame()
restaurants.reset_index(inplace=True)
restaurants.reset_index(inplace=True)
restaurants["index"] = restaurants["index"].apply(lambda s: "burpple_"+ str(s))

In [6]:
print "number of reataurants: %d"%restaurants.shape[0]

number of reataurants: 14455


In [7]:
# map loc to food items
# get a dictonary of {loc:[food_items]}

In [8]:
file_name = "review_with_synsets__20180207.p"
file_name = os.path.join(review_path, file_name)
loc_food = f.retrive_file(file_name)

retrived: /home/yueliu/Desktop/workspace_yue/review_with_synsets__20180207.p


In [9]:
from collections import defaultdict
loc_food_ref = defaultdict(set)
def get_loc_food_ref(line):
    food_item = line["food_items"]
    for loc in line["loc"]:
        loc_food_ref[loc].update([food_item])
loc_food.apply(get_loc_food_ref,1)
print "number of food review loc with food items: %d"%len(loc_food_ref)

number of food review loc with food items: 271124


In [10]:
def map_food(loc_lst):
    food_lst = []
    for loc in loc_lst:
        try:
            food_lst.append(list(loc_food_ref[loc]))
        except:
             pass 
    food_lst =  [item for sublist in food_lst  for item in sublist]
    return sorted([(x,food_lst.count(x)) for x in set(food_lst)], key=lambda x: (-x[1], x))

restaurants["restaurants.food"] = restaurants["loc"].apply(map_food)

In [11]:
restaurants.sample(5)

,index,restaurant.name,loc,restaurants.food
827,burpple_827,Artisan Boulangerie Co. (The Metropolis),"[burppleinitial/review/2SDrwA81, burppleinitia...","[(and, 3), (for, 2), (ham and cheese, 2), (san..."
8022,burpple_8022,Mr Bean (Outram Park MRT),[burppleinitial/review/ATTt4Ft6],"[(hazelnut praline, 1)]"
8603,burpple_8603,OTC Cafe,"[burpple/review/3LkNIriY, burpple/review/E6_Wt...","[(for, 12), (and, 6), (food, 6), (good, 5), (s..."
10425,burpple_10425,Select Cafeteria @ CHervons,[burppleinitial/review/tLHIEjPP],[]
6275,burpple_6275,Killiney Caf (Marina Square),"[burppleinitial/review/-BRQ2mzW, burppleinitia...","[(mee rebus, 2), (and, 1), (food, 1), (for, 1)..."


In [12]:
from copy import deepcopy
restaurant_entities = deepcopy(restaurants)

## 2. Restaurants from delivery sites
    attributes
>     1. Delivery sites locs
    2. Food items offered: (sorted list of food_name & count of delivery sites)

In [13]:
file_name = "all_food_vendor__standard__20180207.p"
#file_name = os.path.join(review_path, file_name)
df_retrived = f.retrive_file(file_name)

retrived: all_food_vendor__standard__20180207.p


In [14]:
print df_retrived.shape
print df_retrived.columns.tolist()
df_retrived.head(1)

(333572, 7)
['food_name', 'vendor_name', 'locs', 'clean_name', 'lemma_name', 'lemma_name_2', 'standard_name']


,food_name,vendor_name,locs,clean_name,lemma_name,lemma_name_2,standard_name
4,Japanese Pickle Roll (6 pcs),Sen of Japan,"[(2017-07-03T17:55:01.668428, deliveroo/food/7...",japanese pickle roll,japanese pickle roll,japanese pickle roll,japanese pickle roll


In [15]:
import pandas as pd
pd.options.mode.chained_assignment = None # default is warn
cols = ["vendor_name", "locs", "standard_name"]
df = df_retrived[cols]
df.rename(index=str, columns={"vendor_name": "restaurant.name"}, inplace=True)

def zip_food_loc(line):
    return [line["standard_name"],line["locs"]]
df["loc"] = df.apply(zip_food_loc, 1)

In [16]:
restaurants = df.groupby('restaurant.name')['loc'].apply(list).to_frame()
restaurants.reset_index(inplace=True)
restaurants.reset_index(inplace=True)
restaurants["index"] = restaurants["index"].apply(lambda s: "delivery_"+ str(s))

def count_food(lst):
    food_lst = []
    for p in lst:
        food_name = p[0]
        locs = p[1]
        delivery_sites = set()
        for loc in locs:
            delivery_sites.add(loc[1].split("/")[0])
        food_lst.append((food_name, len(delivery_sites)))
    return sorted(food_lst, key=lambda x: (-x[1], x))   
restaurants["restaurants.food"] = restaurants["loc"].apply(count_food)

In [17]:
# confirm indeed some food has more than 1 occurance
l = restaurants["restaurants.food"].tolist()
flat_list = [item for sublist in l for item in sublist]
selected = [i for i in flat_list if i[1]>1]
print "Some food is offered by some restaurant on multiple websites: %s"%(len(selected) > 0)

Some food is offered by some restaurant on multiple websites: True


In [18]:
restaurants.head(2)

,index,restaurant.name,loc,restaurants.food
0,delivery_0,,"[[trivento bird bee red mendoza argentina, [('...","[(aglio aglio, 1), (albarino ria baixas, 1), (..."
1,delivery_1,Aloha Pok,"[[aloha standard nalu tuna, [('2017-07-03T17:5...","[(aloha poke standard nalu salmon, 1), (aloha ..."


In [19]:
# extract loc to be list
def extract_loc(lst):
    all_locs = []
    for food in lst:
        food_locs = food[1]
        for l in food_locs:
            all_locs.append(l[1])
    return all_locs
restaurants["loc"] = restaurants["loc"].apply(extract_loc)

In [20]:
restaurants.sample(5)

,index,restaurant.name,loc,restaurants.food
5916,delivery_5916,Wang Thai Kitchen,"[foodpanda/menu_item/AWAMUOk_vFWOHWF12xDN, foo...","[(chicken in dry curry, 2), (clay pot tang hoo..."
2625,delivery_2625,Killiney (Lor Telok),"[foodpanda/menu_item/AV1B4BOWcSlXBuMN34yS, foo...","[(belacan fry rice, 1), (burger, 1), (chee che..."
2416,delivery_2416,Jalan Kayu Prata Cafe (18 Tai Seng),"[foodpanda/menu_item/AV1BocNTcSlXBuMN3hvT, foo...","[(aloo, 1), (aloo mutter masala, 1), (banana p..."
5259,delivery_5259,Teuscher,"[deliveroo/food/32988, deliveroo/food/AV06ww6t...","[(piece bailey truffle box, 1), (piece lake of..."
545,delivery_545,Basil Thai Kitchen - Paragon,"[deliveroo/food/90364, deliveroo/food/AV066Dmb...","[(basil beef with jasmine rice, 1), (basil bee..."


## 3. Merge records

In [21]:
restaurant_entities = pd.concat([restaurant_entities, restaurants],ignore_index=True)

In [22]:
restaurant_entities["food_cnt"] = restaurant_entities["restaurants.food"].apply(len)

In [23]:
print "%d restaurant entities with %d names"%(restaurant_entities.shape[0],
                                              restaurant_entities["restaurant.name"].nunique())
print "%d restaurant entities with food items"%(restaurant_entities[
    restaurant_entities["food_cnt"]>0].shape[0])

20857 restaurant entities with 20013 names
20304 restaurant entities with food items


In [24]:
restaurant_entities.sample(5)

,index,restaurant.name,loc,restaurants.food,food_cnt
17886,delivery_3431,Myra's Beach Club,"[what_to_eat/food/AVx3K6_1cSlXBuMNSitF, what_t...","[(aglio aglio beef, 1), (aglio aglio chicken, ...",97
10381,burpple_10381,Seafood Park Tze Char (Tampines),"[burppleinitial/review/04KdkUft, burppleinitia...","[(for, 2), (beef dry hor fun, 1), (crispy, 1),...",10
5330,burpple_5330,Hup Choon Seafood,"[burppleinitial/review/-OvAjVWY, burpple/revie...","[(for, 3), (and, 2), (food, 2), (all in, 1), (...",18
20329,delivery_5874,Victory Boat Noodle Cafe,"[what_to_eat/food/AV9zNptrvFWOHWF1XFqz, what_t...","[(beef boat noodle, 1), (boil beef ball, 1), (...",33
14873,delivery_418,Authentic Mun Chee Kee King of Pig's Organ Soup,"[what_to_eat/food/AVx3JkoYcSlXBuMNSgw2, what_t...","[(braise big intestine, 1), (braise pork rice,...",16


In [25]:
file_name = "__".join(["restaurant_entities", f.today()]) + ".p"
f.save_file(restaurant_entities, file_name)

saved: restaurant_entities__20180207.p
